In [2]:
import gymnasium as gym 
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import os
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [14]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [15]:
env = ShowerEnv()

C:\Users\josh\anaconda3\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [35]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

IndexError: invalid index to scalar variable.

In [28]:
env.close()

In [73]:
env = make_atari_env('Breakout-v0', n_envs=16, seed=0)
env = VecFrameStack(env, n_stack=16)

In [16]:
model = A2C("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [18]:
model.learn(total_timesteps=10000)

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 60        |
|    ep_rew_mean        | -28       |
| time/                 |           |
|    fps                | 397       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -0.115    |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 5353      |
|    policy_loss        | -0.0255   |
|    value_loss         | 1.82      |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 60       |
|    ep_rew_mean        | -16.4    |
| time/                 |          |
|    fps                | 389      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/             

In [5]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C_2M_model')

In [68]:
model.save(a2c_path)

In [8]:
env = make_atari_env('Breakout-v0', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

C:\Users\josh\anaconda3\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment Breakout-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


In [9]:
model = A2C.load(a2c_path, env)

C:\Users\josh\anaconda3\Lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
C:\Users\josh\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


Wrapping the env in a VecTransposeImage.


C:\Users\josh\anaconda3\Lib\site-packages\stable_baselines3\common\base_class.py:748: UserWarning: You are probably loading a model saved with SB3 < 1.7.0, we deactivated exact_match so you can save the model again to avoid issues in the future (see https://github.com/DLR-RM/stable-baselines3/issues/1233 for more info). Original error: Error(s) in loading state_dict for ActorCriticCnnPolicy:
	Missing key(s) in state_dict: "pi_features_extractor.cnn.0.weight", "pi_features_extractor.cnn.0.bias", "pi_features_extractor.cnn.2.weight", "pi_features_extractor.cnn.2.bias", "pi_features_extractor.cnn.4.weight", "pi_features_extractor.cnn.4.bias", "pi_features_extractor.linear.0.weight", "pi_features_extractor.linear.0.bias", "vf_features_extractor.cnn.0.weight", "vf_features_extractor.cnn.0.bias", "vf_features_extractor.cnn.2.weight", "vf_features_extractor.cnn.2.bias", "vf_features_extractor.cnn.4.weight", "vf_features_extractor.cnn.4.bias", "vf_features_extractor.linear.0.weight", "vf_featu

In [21]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action, _states = model.predict(obs, deterministic=False)
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:60
Episode:2 Score:-60
Episode:3 Score:-60
Episode:4 Score:-60
Episode:5 Score:-60
Episode:6 Score:-60
Episode:7 Score:-60
Episode:8 Score:-60
Episode:9 Score:-60
Episode:10 Score:60


In [19]:
vec_env = model.get_env()
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = vec_env.step(action)
    print(obs)

[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[35.]]
[[36.]]
[[36.]]
[[36.]]
[[36.]]
[[36.]]
[[36.]]


KeyboardInterrupt: 

In [13]:
env.close()

NameError: name 'env' is not defined